In [1]:
# HIDE CODE
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
matplotlib.rcParams.update({'font.size': 12})

import warnings
import sys
import os
sys.path.append('/home/server/gli-data-science/')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

import os
import ds_db
import helper_db
from helper import transform_to_rupiah, rupiah_format

import pickle
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display, HTML, display_html, IFrame
import ipywidgets as ipyw


    
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RANSACRegressor, RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, export_text, DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVR, SVC, LinearSVC
import xgboost as xgb 


from joblib import dump, load
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split

### ide kasar
> 
* daily job to loop all member alfagift 10 juta
* compute feature trx voucher used, # of trx, # of sales
* compute feature event app site open / view product / atc
* updating each feature and re predict churn feature
* lets play in month - 3 for each define trx and event feature

In [2]:
str_now = datetime.now()
str_now_month = str_now.strftime('%b%y')

In [3]:
df_pv_ = pd.read_parquet('./feature/pv_{}'.format(str_now_month))
df_pv_['EVENT_TIME'] = pd.to_datetime(df_pv_['EVENT_TIME']).dt.strftime('%Y-%m')

df_pv = df_pv_.groupby(['TRO_PONTA_ID','EVENT_TIME']).agg(PV=('COUNT_VIEW_PRODUCT','sum'))\
        .unstack(level=1).fillna(0).reset_index()

df_pv.columns = [' '.join(col).strip().upper() for col in df_pv.columns.values]

In [4]:
df_pv.head()

,TRO_PONTA_ID,PV 2022-06,PV 2022-07,PV 2022-08,PV 2022-09
0,9990010000001200,5.0,1.0,3.0,2.0
1,9990010000001771,2.0,8.0,2.0,0.0
2,9990010000001999,1.0,4.0,1.0,0.0
3,9990010000004582,1.0,2.0,21.0,0.0
4,9990010000006038,0.0,0.0,0.0,6.0


In [5]:
df_re_g_u = pd.read_parquet('./feature/rating_{}'.format(str_now_month))
df_re_g_u = df_re_g_u.rename(columns={'TRO_MEMBERS':'TRO_PONTA_ID'})

In [6]:
df_re_g_u.head()

,TRO_PONTA_ID,RATING_BELOW 2022-06,RATING_BELOW 2022-07,RATING_BELOW 2022-08,RATING_BELOW 2022-09,RATING_AVG 2022-06,RATING_AVG 2022-07,RATING_AVG 2022-08,RATING_AVG 2022-09
0,9990010000001200,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1,9990010000006038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,9990010000084000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,9990010000090310,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0
4,9990010000102923,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [7]:
# HIDE CODE

## USING ORDER
import glob

df_tto = []
for end_date in pd.date_range(str_now - relativedelta(months=4), str_now, freq='M'):
    start_date = end_date.replace(day=1)
    end_date_save = start_date + relativedelta(months=1)
    
    start_date_str = start_date.strftime('%d%b%y')
    end_date_str = end_date_save.strftime('%d%b%y')
    
    print(start_date_str, end_date_str)
    
    pf = '/home/server/gli-data-science/data/trans_oshop/{}_{}.parquet'.format(start_date_str, end_date_str)
    df_tto.append(pd.read_parquet(pf, engine='fastparquet'))
    
df_tto = pd.concat(df_tto)

# df_tto = df_tto[df_tto['TRO_MEMBERS'] != 'N']
df_tto['TRO_DATE'] = pd.to_datetime(df_tto['TRO_DATE'])
df_tto['TRO_NET'] = df_tto['TRO_NET'].astype('float', errors='ignore').astype('int', errors='ignore')


df_tto['TRO_QTY'] = df_tto['TRO_QTY'].astype(int)




01Jun22 01Jul22
01Jul22 01Aug22
01Aug22 01Sep22
01Sep22 01Oct22


In [8]:
## USE THIS FOR GET PV DATA FROM DUMP DATA FILE (SADDDDD ALWAYS DEADLINE)

df_mem_push = df_tto[['TRO_PONTA_ID']].drop_duplicates()
from sqlalchemy import event,create_engine,types


engine_stmt = "oracle://%s:%s@%s/%s" % ('report', 'justd0it', '10.234.152.61', 'alfabi' )
engine = create_engine(engine_stmt)

df = df_mem_push.astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()

### USE THIS FOR GET PV DATA FROM DUMP DATA FILE (SADDDDD ALWAYS DEADLINE)

# from google.oauth2.service_account import Credentials
# import pandas_gbq

# credentials = Credentials.from_service_account_file(
#     '/home/server/gli-data-science/akhiyar/lib/gli-dw-alfagift-admin.json')
# pandas_gbq.context.credentials = credentials

# #TODO: Set project_id to your Google Cloud Platform project ID.
# project_id = "gli-dw-alfagift"

# #TODO: Set table_id to the full destination table ID (including the
# #      dataset ID)
# private_key = '/home/server/gli-data-science/akhiyar/lib/gli-dw-alfagift-admin.json'

# table_id = 'data_science.temp_churn'
# pandas_gbq.to_gbq(
# df_mem_push,
# table_id,
# project_id=project_id,
# if_exists = 'replace')

In [9]:
str_from = ((str_now - relativedelta(months=4)).replace(day=1)).strftime('%Y-%m-%d')
str_until = ((str_now.replace(day=1)) - relativedelta(days=1)).strftime('%Y-%m-%d')
print(str_from, str_until)

2022-06-01 2022-09-30


In [10]:
cond1 = (df_tto['TRO_DATE'] >= str_from)
cond2 = (df_tto['TRO_DATE'] <= str_until)
df_tto_sel = df_tto[ cond1 & cond2 ]

In [11]:
df_sla = pd.read_parquet('./feature/sla_{}'.format(str_now_month))
df_sla['ATT_SEND_DATE_TOSTORE'] = pd.to_datetime(df_sla['ATT_SEND_DATE_TOSTORE'])
df_sla['SLA'] = np.where(df_sla['SLA_NORM']< 0, 20, df_sla['SLA_NORM'])

df_tto_sel = pd.merge(df_tto_sel, df_sla, left_on='TRO_NO_ORDER', right_on='ATT_ORDER_ID', how='left')

col_drop = [
    'ATT_ORDER_ID',
    'ATT_ORDER_DATE',
    'ATT_DELIVERY_DATE',
    'ATT_SEND_DATE_TOSTORE',
    'SLA_NORM'        
]
df_tto_sel = df_tto_sel.drop(col_drop,1)

## create feature
df_tto_sel['SLA'] = df_tto_sel['SLA'].fillna(20)
df_tto_sel['SLA_LATE'] = np.where(df_tto_sel['SLA'] > 60, 1,0)

In [12]:

# q = '''
# SELECT 
# 	tc.TRO_PONTA_ID, 
# 	(TO_DATE('{}', 'YYYY-mm-dd')) - amc.AMC_REGISTER_DATE AS SINCE_REGIS
# FROM 
# 	TEMP_CHURN tc
# LEFT JOIN ALFAGIFT_MASTER_CUST amc 
# ON amc.AMC_MEMBER = tc.TRO_PONTA_ID 


# '''.format(str_until)


# con = ds_db.connect_alfabi()

# df_regis = pd.read_sql_query(q, con)
# df_regis['SINCE_REGIS'] = df_regis['SINCE_REGIS'].fillna('-1')
# con.close()


df_regis = pd.read_parquet('./feature/regis_oct22')
df_regis['SINCE_REGIS'] = pd.to_datetime(df_regis['SINCE_REGIS'])
# df_regis['SINCE_REGIS_NOW'] = str_now
df_regis['SINCE_REGIS'] = (str_now - df_regis['SINCE_REGIS']).dt.days

In [13]:
df_tto_sel['TRO_DATE_MONTH'] = (df_tto_sel['TRO_DATE'].dt.year.astype(str)
                                + '-0' + df_tto_sel['TRO_DATE'].dt.month.astype(str))

In [ ]:
df_tto_sel_g = df_tto_sel.groupby(['TRO_PONTA_ID','TRO_DATE_MONTH','TRO_NO_ORDER'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_NO_ORDER':'nunique',
                              'SLA':'first',
                              'SLA_LATE':'first',
                              'TRO_PAYMENT_METHOD':'first'
                             })
df_tto_sel_g = df_tto_sel_g.groupby(['TRO_PONTA_ID','TRO_DATE_MONTH'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_NO_ORDER':'sum',
                              'SLA':'sum',
                              'SLA_LATE':'sum',
                              'TRO_PAYMENT_METHOD':'nunique'
                             })

In [ ]:
df_tto_sel_g.head()

### unstack all feature in oshop

In [ ]:
df_tto_sel_g_u = df_tto_sel_g.unstack(level=1).fillna(0).reset_index()
df_tto_sel_g_u.columns = [' '.join(col).strip().upper() for col in df_tto_sel_g_u.columns.values]

## merge with rating

In [ ]:
df_re_g_u['TRO_PONTA_ID'] = df_re_g_u['TRO_PONTA_ID'].astype(str)
df_tto_re = pd.merge(df_tto_sel_g_u, df_re_g_u, on='TRO_PONTA_ID', how='left').fillna(0)

## merge with product view

In [ ]:
df_pv['TRO_PONTA_ID'] = df_pv['TRO_PONTA_ID'].astype(str)
df_tto_re_pv = pd.merge(df_tto_re, df_pv, on='TRO_PONTA_ID', how='left').fillna(0)

## merge with regis

In [ ]:
df_tto_re_pv = pd.merge(df_tto_re_pv, df_regis, on='TRO_PONTA_ID')
df_tto_re_pv = df_tto_re_pv[df_tto_re_pv['SINCE_REGIS'].astype(int) > 0]

In [ ]:
f1 = (str_now-relativedelta(months=1)).strftime('%Y-%m')
f2 = (str_now-relativedelta(months=2)).strftime('%Y-%m')
f3 = (str_now-relativedelta(months=3)).strftime('%Y-%m')
f4 = (str_now-relativedelta(months=4)).strftime('%Y-%m')

In [ ]:
print(f1,f2,f3,f4)

## Random Forest

### not oversampling version without threshold 2/3 or 3/3

In [ ]:
from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET {}'.format(f4)] > 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f3)] > 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f2)] > 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f1)] == 0)) | \
    
((df_tto_re_pv['TRO_NET {}'.format(f4)] > 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f3)] == 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f2)] > 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f1)] == 0)) |
    
((df_tto_re_pv['TRO_NET {}'.format(f4)] == 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f3)] > 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f2)] > 0) & \
(df_tto_re_pv['TRO_NET {}'.format(f1)] == 0))
)

condition_3_3 = ( 
          (df_tto_re_pv['TRO_NET {}'.format(f4)] > 0) & \
          (df_tto_re_pv['TRO_NET {}'.format(f3)] > 0) & \
          (df_tto_re_pv['TRO_NET {}'.format(f2)] > 0) & \
          (df_tto_re_pv['TRO_NET {}'.format(f1)] == 0) 
)

df_tto_re_pv['label'] = np.where(condition_3_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_PONTA_ID',
'TRO_NET {}'.format(f1), 
'TRO_NO_ORDER {}'.format(f1),
'SLA {}'.format(f1),
'SLA_LATE {}'.format(f1),
'RATING_BELOW {}'.format(f1),
'RATING_AVG {}'.format(f1),
'PV {}'.format(f1),
'TRO_PAYMENT_METHOD {}'.format(f1),
'label'
], 1
)
y = df_tto_re_pv[['label']]


# applying standard scaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
dump(scaler, './model/scaler_{}.joblib'.format(str_now_month))


clf_all_mlp = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
clf_all_mlp.fit(X, y) 
dump(clf_all_mlp, './model/mlp_{}.joblib'.format(str_now_month))




In [ ]:
X_12 = df_tto_re_pv.drop(
[
'TRO_PONTA_ID',
'TRO_NET {}'.format(f1), 
'TRO_NO_ORDER {}'.format(f1),
'SLA {}'.format(f1),
'SLA_LATE {}'.format(f1),
'RATING_BELOW {}'.format(f1),
'RATING_AVG {}'.format(f1),
'PV {}'.format(f1),
'TRO_PAYMENT_METHOD {}'.format(f1),
'label'
], 1
)
X_12['SINCE_REGIS'] = X_12['SINCE_REGIS']+30

### do a prediction

In [ ]:
scaler_all = load('./model/scaler_{}.joblib'.format(str_now_month))
clf_all = load('./model/mlp_{}.joblib'.format(str_now_month))


X_12_scale = pd.DataFrame(scaler_all.transform(X_12), index=X_12.index, columns=X_12.columns)
pred = (clf_all.predict_proba(X_12_scale)[:,1] >= 0.52).astype(int)

c_pred = [
    df_tto_re_pv[['TRO_PONTA_ID']].reset_index(drop=True),
    pd.DataFrame(pred, columns=['PRED'])
]
df_mem_12 = pd.concat(c_pred, 1)

In [ ]:
print("PREDICTION RESULT {}".format(df_mem_12[df_mem_12['PRED'] == 1].shape))

In [ ]:
df_trader = pd.read_excel(
    '/home/server/gli-data-science/ferina/Sales Trader Ytd Aug-ExEmployee.xlsx'
    , dtype='object', names=['member','stat']
)

In [ ]:
df_mem_12 = df_mem_12[~df_mem_12['TRO_PONTA_ID'].isin(df_trader['member'])]

In [ ]:
print("AFTER REMOVE TRADER {}".format(df_mem_12[df_mem_12['PRED'] == 1].shape))

In [ ]:
engine = create_engine(engine_stmt)

df = df_mem_12[df_mem_12['PRED'] == 1].astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn_target', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()

In [ ]:
start_q = (str_now.replace(day=1)).strftime('%d-%b-%y')
end_q = (str_now+relativedelta(months=1)).replace(day=1).strftime('%d-%b-%y')

In [ ]:
q = '''

SELECT DISTINCT(tct.TRO_PONTA_ID) AS TRO_PONTA_ID 
FROM TEMP_CHURN_TARGET tct
LEFT JOIN (
	SELECT 
		gua.GUA_ACCOUNT_CARD_ADJ
	FROM GLI_UNINSTALL_ALFAGIFT gua 
	WHERE 
		gua.GUA_STATUS = 'Y'
		AND trunc(GUA_UNINSTALL_DATE) BETWEEN '01-aug-21' AND '{}'
)gua
ON gua.GUA_ACCOUNT_CARD_ADJ = tct.TRO_PONTA_ID 
LEFT JOIN (
	SELECT 
		rto.TRO_PONTA_ID
	FROM 
		rpt_trans_oshop rto
	WHERE 
		rto.TRO_DATE BETWEEN '{}' AND '{}'
)rto
ON rto.TRO_PONTA_ID = tct.TRO_PONTA_ID 
LEFT JOIN (
	SELECT 
		amc.AMC_MEMBER
	FROM 
		ALFAGIFT_MASTER_CUST amc
	WHERE 
		amc.AMC_FLAG_FRAUD = 'Y'
)amc
ON amc.AMC_MEMBER = tct.TRO_PONTA_ID 
WHERE gua.GUA_ACCOUNT_CARD_ADJ IS NULL
AND rto.TRO_PONTA_ID IS NULL
AND amc.AMC_MEMBER IS NULL

'''.format(end_q, start_q, end_q)


con = ds_db.connect_alfabi()
df_churn_sel = pd.read_sql_query(q, con)
con.close()


In [ ]:
df_churn_sel['PERIOD'] = str_now.replace(day=1).strftime('%Y-%m-%d')
df_churn_sel['PERIOD'] = pd.to_datetime(df_churn_sel['PERIOD'])
df_churn_sel = df_churn_sel[['PERIOD','TRO_PONTA_ID']].rename(columns={'TRO_PONTA_ID':'MEMBER'})
df_churn_all = df_churn_sel.copy()

In [ ]:
print('HASIL AKHIR {}'.format(df_churn_all.shape))

In [ ]:


from sqlalchemy import event,create_engine,types
engine_alfabi_smt = "oracle://%s:%s@%s/%s" % ( 'report', 'justd0it', '10.234.152.61', 'alfabi' )
    
engine = create_engine(engine_alfabi_smt)

q = '''

DELETE FROM ALFAGIFT_CHURN
WHERE PERIOD = '{}'

'''.format(str_now.replace(day=1).strftime('%d-%b-%y'))

con = engine.connect()
con.execute(q)

df = df_churn_all.copy()
dtype = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('ALFAGIFT_CHURN', engine, index=False, if_exists="append", dtype=dtype)



engine.dispose()

In [ ]:
df_churn_view = pd.merge(
    df_churn_sel,df_tto_re_pv,
    left_on='MEMBER',
    right_on='TRO_PONTA_ID',
    how='left'
)
df_churn_view.head(20)